In [238]:
import pandas as pd
import pprint
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 25, 13

In [239]:
from pymongo import MongoClient

from IPython.core.display import display, HTML
[display(HTML("<style>.container { width:100% !important; }</style>"))];

## Überschrift

In [240]:
with MongoClient('mongodb://localhost:27017/') as client:
    database = client["dwh"]
    aggregated_data = database.aggregated_data
    country_codes = database.country_codes
    medal_wins = database.medal_wins
    who_ppp_2017 = database.who_ppp_2017

In [241]:
def out(Country):
    #return list(aggregated_data.find({"country": Country}))
    #values = list(aggregated_data.find({"country": 'Germany'}))[0]['medal_wins']['gold']['by_year']
    data = aggregated_data.find({"country": Country})
    x_values, y_values = get_medals(data)
    show_graph(x_values, y_values)
    return 0

In [242]:
data = list(aggregated_data.find({"country": 'Germany'}))

In [243]:
x_values, y_values = get_medals(data)

In [244]:
def get_medals(data):
    # Get all gold medals, sorting the dict
    dict_gold = data[0]['medal_wins']['gold']['by_year']
    dict_gold = {k: dict_gold[k] for k in sorted(dict_gold.keys(), key=lambda x: int(x))}
    
    # Get all silver medals, sorting the dict
    dict_silver = data[0]['medal_wins']['silver']['by_year']
    dict_silver = {k: dict_silver[k] for k in sorted(dict_silver.keys(), key=lambda x: int(x))}
    
    # Get all bronze medals, sorting the dict
    dict_bronze = data[0]['medal_wins']['bronze']['by_year']
    dict_bronze = {k: dict_bronze[k] for k in sorted(dict_bronze.keys(), key=lambda x: int(x))}
    
    
    # Defining x and y values for the graph
    years = {}
    for year in sorted(set(dict_gold.keys()) | set(dict_silver.keys()) | set(dict_bronze.keys())):
        years[year] = dict_gold.get(year, 0) + dict_silver.get(year, 0) + dict_bronze.get(year, 0)

    #x_values = years.keys()
    x_values = []
    y_values = []
    for x in years:
        x_values += [x]
        y_values += [years[x]]
        
    return x_values, y_values

In [250]:
def get_expenditure(data):
    return data[0]['health_expenditures']

In [262]:
test_data = get_expenditure(data)
per_capita = test_data['per_capita']
absolute = test_data['absolute']
print('Per Capita: ' + str(per_capita))
print('Absolute: ' + str(absolute))

Per Capita: 488745.8977
Absolute: 5922.63824


In [245]:
def show_graph(x_values, y_values):
    df=pd.DataFrame({'x_values': x_values, 'y_values': y_values })
    
    # plot
    plt.plot( 'x_values', 'y_values', data=df)
    plt.show()

In [246]:
interact(out, Country=[cc['country'] for cc in country_codes.find()]);

interactive(children=(Dropdown(description='Country', options=('Ukraine', 'Jordan', 'Macedonia', 'Slovakia', '…